<a href="https://colab.research.google.com/github/ataata107/dystab/blob/master/createDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/philferriere/tfoptflow.git

In [ ]:
%cd tfoptflow/tfoptflow/

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.13.2
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==1.13.2

Download the FBMS dataset and place it in this folder to be zipped in next step

In [ ]:
!unzip FBMS_Trainingset.zip

In [ ]:
from __future__ import absolute_import, division, print_function
from copy import deepcopy
from skimage.io import imread,imsave
from skimage.transform import rescale, resize, downscale_local_mean
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TEST_OPTIONS
from visualize import display_img_pairs_w_flows
import numpy as np

In [ ]:
# TODO: Set device to use for inference
# Here, we're using a GPU (use '/device:CPU:0' to run inference on the CPU)
gpu_devices = ['/device:GPU:0']  
controller = '/device:GPU:0'

# TODO: Set the path to the trained model (make sure you've downloaded it first from http://bit.ly/tfoptflow)
ckpt_path = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'

# Configure the model for inference, starting with the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TEST_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = ckpt_path
nn_opts['batch_size'] = 1
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# We're running the PWC-Net-large model in quarter-resolution mode
# That is, with a 6 level pyramid, and upsampling of level 2 by 4 in each dimension as the final flow prediction
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# The size of the images in this dataset are not multiples of 64, while the model generates flows padded to multiples
# of 64. Hence, we need to crop the predicted flows to their original size
nn_opts['adapt_info'] = (1, 436, 1024, 2)

# Instantiate the model in inference mode and display the model configuration
nn = ModelPWCNet(mode='test', options=nn_opts)
nn.print_config()

In [ ]:
import os
import cv2
img_dirs = sorted(os.listdir("./Trainingset/"))
print(img_dirs)
if not os.path.exists('./Dataset'):
  os.makedirs('./Dataset')
if not os.path.exists('./Dataset/image_pair'):
    os.makedirs('./Dataset/image_pair')
if not os.path.exists('./Dataset/flow_pair'):
    os.makedirs('./Dataset/flow_pair')
if not os.path.exists('./Dataset/flows'):
    os.makedirs('./Dataset/flows')
counter = 0
for i in img_dirs:
  print(i)
  
  imgs = sorted(os.listdir("./Trainingset/"+i+"/"))
  imgs = (list(filter(lambda x: x!="GroundTruth" and not x.endswith(".bmf"), imgs)))
  # print(len(imgs))
  for j in range(len(imgs)-3):
    img_pairs = []
    img_names = []  
    img_pairs_back = []
    image_path1 = "./Trainingset/"+i+"/" + imgs[j]
    image_path2 = "./Trainingset/"+i+"/" + imgs[j+1]
    image1, image2 = cv2.imread(image_path1), cv2.imread(image_path2)
    image1, image2 = cv2.resize(image1, (256, 256)),cv2.resize(image2, (256, 256))

    img_pairs.append((image1, image2))
    img_pairs_back.append((image2, image1))
    img_names.append((imgs[j], imgs[j+1]))

    image_path2 = "./Trainingset/"+i+"/" + imgs[j+2]
    image2 = cv2.imread(image_path2)
    image2 = cv2.resize(image2, (256, 256))
    img_pairs.append((image1, image2))
    img_pairs_back.append((image2, image1))
    img_names.append((imgs[j], imgs[j+2]))

    image_path2 = "./Trainingset/"+i+"/" + imgs[j+3]
    image2 = cv2.imread(image_path2)
    image2 = cv2.resize(image2, (256, 256))
    img_pairs.append((image1, image2))
    img_pairs_back.append((image2, image1))
    img_names.append((imgs[j], imgs[j+3]))


    
    pred_labels_front = nn.predict_from_img_pairs(img_pairs, batch_size=1, verbose=False)
    pred_labels_back = nn.predict_from_img_pairs(img_pairs_back, batch_size=1, verbose=False)
    for k in range(len(img_pairs)):
      if not os.path.exists('./Dataset/image_pair/'+ str(k+counter)):
        os.makedirs('./Dataset/image_pair/'+ str(k+counter))
      if not os.path.exists('./Dataset/flow_pair/'+ str(k+counter)):
        os.makedirs('./Dataset/flow_pair/'+ str(k+counter))
      cv2.imwrite("./Dataset/image_pair/"+ str(k+counter) + "/" + img_names[k][0],img_pairs[k][0])
      cv2.imwrite("./Dataset/image_pair/"+ str(k+counter) + "/" + img_names[k][1],img_pairs[k][1])
      np.save("./Dataset/flow_pair/"+ str(k+counter) + "/forward", pred_labels_front[k])
      np.save("./Dataset/flow_pair/"+ str(k+counter) + "/backward", pred_labels_back[k])
    
      if not os.path.exists('./Dataset/flows/'+ str(k+counter)):
        os.makedirs('./Dataset/flows/'+ str(k+counter))
      for o,l in enumerate(pred_labels_front):
        np.save("./Dataset/flows/"+ str(k+counter) + "/"+str(o), l)
    counter+=3
    print(counter)
  
    
  

Now move the Dataset folder created here to the dystab root folder